# 0.0 Import Libs

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import sqlite3

from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from datetime import datetime

# 1.0 Collect all items in main page

In [7]:
#========== Create request and BeautifulSoup ==========#
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get( url, headers=headers )
soup = BeautifulSoup(page.text, 'html.parser')

# Realizando a request do site, utilizando um agent simulando um browser para evitar problemas com a requisição
# Basicamente, você utiliza o request para requisitar os dados do url simulando um browser
## E instância o texto para o BeautifulSoup, para realizar a extração dos dados HTML
### O parser é a forma que o BeautifulSoup vai ler os dados do HTML

# get pagesize max to extract all items
total_item = soup.find('h2', class_='load-more-heading')
total_item = total_item.get('data-total')
page_number = np.round(int(total_item)/36)

# create new request to extract all items
url01 = url + '?sort=stock&image-size=small&image=model&offset=0&page-size=' + str(int(page_number*36))
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get( url01, headers=headers )
soup = BeautifulSoup(page.text, 'html.parser')

# idxing product list
products = soup.find('ul', class_='products-listing small')
product_list = products.find_all('article', class_='hm-product-item')

# collect all products id
product_id = [p.get('data-articlecode') for p in product_list]

# collect all products category
product_category = [p.get('data-category') for p in product_list]

# collect all products name
product_list = products.find_all('a', class_='link')
product_name = [p.get_text() for p in product_list]

# collect all products price
product_list = products.find_all('span',class_='price regular')
product_price = [p.get_text() for p in product_list]

# transform all collects in dataframe
data = pd.DataFrame([product_id, product_name, product_category,product_price]).T
data.columns = ['product_id', 'product_name', 'product_category','product_price']

# scrapy datetime
data['scrapy-datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## 1.1 Identify data details of an items to later collect all details of all items

In [ ]:
# get color and composition one product - API request
url = 'https://www2.hm.com/en_us/productpage.0985197001.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get( url, headers=headers )

# BeautifulSoup Object
soup = BeautifulSoup(page.text, 'html.parser')

#=======================color name====================#
# A primeira cor, quando esta selecionada a classe altera para active, neste sentido, criei 2 modos e coleta
product_list = soup.find_all('a', class_='filter-option miniature active')
color_name = [p.get('data-color') for p in product_list]

product_list = soup.find_all('a', class_='filter-option miniature')
color_name2 = [p.get('data-color') for p in product_list]

color_name = color_name + color_name2

# color id
product_list = soup.find_all('a', class_='filter-option miniature active')
color_id1 = [p.get('data-articlecode') for p in product_list]

product_list = soup.find_all('a', class_='filter-option miniature')
color_id2 = [p.get('data-articlecode') for p in product_list]

color_id = color_id1 + color_id2

df_color = pd.DataFrame([color_id, color_name]).T
df_color.columns = ['product_id','color_name']

# generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

#======================= composition ====================#
product_composistion_list = soup.find_all('div', class_='pdp-description-list-item')
product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composistion_list]

# rename dataframe
df_composition = pd.DataFrame(product_composition).T
df_composition.columns = df_composition.iloc[0]

# drop first row
df_composition = df_composition.iloc[1:].fillna(method='ffill')
df_composition

# generate style id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition

#======================= merge ====================#
data_sku = pd.merge(df_color, df_composition[['style_id','Fit','Composition']], how='left', on='style_id')

## 1.2 Multi products scrapy

In [61]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

#empty dataframe
data_details = pd.DataFrame()

#unique columns for all products composition
aux = []
cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']

df_pattern = pd.DataFrame(columns=cols)

for i in range(len(data)):
    # Get color and composition one product - API request
    url = 'https://www2.hm.com/en_us/productpage.'+ data.loc[i, 'product_id'] +'.html'
    page = requests.get( url, headers=headers )

    # BeautifulSoup Object
    soup = BeautifulSoup(page.text, 'html.parser')

    #=======================color name====================#
    # A primeira cor, quando esta selecionada a classe altera para active, neste sentido, criei 2 modos e coleta
    product_list = soup.find_all('a', class_=['filter-option miniature active','filter-option miniature'])
    color_name = [p.get('data-color') for p in product_list]

    # color id
    product_list = soup.find_all('a', class_=['filter-option miniature active','filter-option miniature'])
    color_id = [p.get('data-articlecode') for p in product_list]

    df_color = pd.DataFrame([color_id, color_name]).T
    df_color.columns = ['product_id','color_name']

    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

    #======================= composition ====================#
    product_composistion_list = soup.find_all('div', class_='pdp-description-list-item')
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composistion_list]

    # rename dataframe
    df_composition = pd.DataFrame(product_composition).T
    df_composition.columns = df_composition.iloc[0]

    # drop first row
    df_composition = df_composition.iloc[1:].fillna(method='ffill')

    # garantee the same number of columns
    df_compostion = pd.concat([df_pattern, df_composition], axis=0)
    
    # generate style id
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    
    aux = aux + df_composition.columns.tolist()
    
    #======================= merge ====================#
    data_sku = pd.merge(df_color, df_composition, how='left', on='style_id')
    
    # all products details
    data_details = pd.concat([data_details, data_sku], axis=0)

# join data showroom + data details
data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
data_raw = pd.merge(data, data_details[['style_id','color_id','color_name','Fit','Composition','Product safety','Size']], how='left', on='style_id')

In [313]:
data_raw.to_csv('data-raw.csv', index=False)

NameError: name 'data_raw' is not defined

# 2.0 Data cleaning

In [41]:
#===================== Data Cleaning =====================# 
data = pd.read_csv('data-raw.csv')

df = data.copy()
df.columns = [item.lower() for item in df.columns]

#product_id
# product_name
df['product_name'] = df['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_category
df['product_category'] = df['product_category'].fillna('no_category') 

# product_price
df['product_price'] = df['product_price'].apply(lambda x: x.replace('$ ','')).astype(float)

# color_name
df['color_name'] = df['color_name'].apply(lambda x: x.replace(' ', '_').lower())

# fit
df['fit'] = df['fit'].apply(lambda x: x.replace(' ', '_').lower())

# size
df['size_number'] = df['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
df['size_number'] = df['size_number'].apply(lambda x: re.search('\d{3}', x).group(0) if pd.notnull(x) else x)

df['size_model'] = df['size'].str.extract('(\d+/\\d+)')

# composition
df = df[~df['composition'].str.contains('Pocket lining:', na=False)]
df = df[~df['composition'].str.contains('Lining:', na=False)]
df = df[~df['composition'].str.contains('Shell:', na=False)]

# drop duplicates
df = df.drop_duplicates(subset=['product_id', 'product_name', 'product_category', 'product_price',
                                'scrapy-datetime', 'style_id', 'color_id', 'color_name', 'fit'], keep='last')

# reset Index
df = df.reset_index(drop=True)
            
#============= Break Composition Comma ==================#
df1 = df['composition'].str.split(',', expand=True)
df_ref = pd.DataFrame(index=np.arange(len(df)), columns=['cotton', 'polyester', 'elastane','elasterell'])

# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'
df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'
df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('Polyester 0%')

# elastano
df_elastano = df1.loc[df1[1].str.contains('Elastane', na=True), 1]
df_elastano.name = 'elastane'

# combine elastane from colums 1 and 2 df_ref
df_elastano = df_elastano.combine_first(df1[2])

df_ref = pd.concat([df_ref, df_elastano], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elastane'] = df_ref['elastane'].fillna('Elastane 0%')

# elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na=True), 1]
df_elasterell.name = 'elasterell'
df_ref = pd.concat([df_ref, df_elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['elasterell'] = df_ref['elasterell'].fillna('Elasterrel-P 0%')

# final join
df = pd.concat([df, df_ref], axis=1, join='inner')

# format composition data
df['cotton'] = df['cotton'].apply(lambda x: int( re.search('\d+',x).group(0)) / 100 if pd.notnull(x) else x)
df['polyester'] = df['polyester'].apply(lambda x: int( re.search('\d+',x).group(0)) / 100 if pd.notnull(x) else x)
df['elastane'] = df['elastane'].apply(lambda x: int( re.search('\d+',x).group(0)) / 100 if pd.notnull(x) else x)
df['elasterell'] = df['elasterell'].apply(lambda x: int( re.search('\d+',x).group(0)) / 100 if pd.notnull(x) else x)

# drop Columns
df = df.drop(columns=['size','product safety','composition'], axis=1)

# columns rename
df.columns = ['product_id', 'product_name', 'product_category', 'product_price',
       'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'fit',
       'size_number', 'size_model', 'cotton', 'polyester', 'elastane',
       'elasterell']

df = df.drop_duplicates()
df.shape

(496, 15)

In [42]:
df.to_csv('data-cleaned.csv', index=False)

# 3.0 Create database

In [43]:
# create database
data = pd.read_csv('data-cleaned.csv')

query_schema = """
    CREATE TABLE showroom(
        product_id         INTERGER,
        product_name       TEXT, 
        product_category   TEXT,
        product_price      REAL,
        scrapy_datetime    TEXT,
        style_id           INTERGER,
        color_id           INTERGER,
        color_name         TEXT,
        fit                TEXT,
        size_number        REAL,
        size_model         TEXT,
        cotton             REAL,
        polyester          REAL, 
        elastane           REAL,
        elasterell         REAL
    )
"""

conn = sqlite3.connect('hm_db.sqlite')

# execute query schema
conn = sqlite3.connect('hm_db.sqlite')
cursor = conn.execute(query_schema)
conn.commit()

# insert dataset in database
data.to_sql('showroom', con=conn, if_exists='append', index=False)

In [2]:
# connect to db and execute query's
conn = create_engine('sqlite:///hm_db.sqlite', echo=False)

In [20]:
query = """
    SELECT *
    FROM showroom
"""

# read db
df = pd.read_sql_query(query, conn)
df.head()

,product_id,product_name,product_category,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,polyester,elastane,elasterell
0,427159006,trashed_skinny_jeans,men_jeans_ripped,39.99,2021-07-18 18:24:31,427159,1,black_denim,skinny_fit,184.0,31/32,0.93,0.06,0.01,0.0
1,427159006,trashed_skinny_jeans,men_jeans_ripped,39.99,2021-07-18 18:24:31,427159,2,blue_washed_out,skinny_fit,184.0,31/32,0.93,0.06,0.01,0.0
2,427159006,trashed_skinny_jeans,men_jeans_ripped,39.99,2021-07-18 18:24:31,427159,3,denim_blue,skinny_fit,184.0,31/32,0.93,0.06,0.01,0.0
3,427159006,trashed_skinny_jeans,men_jeans_ripped,39.99,2021-07-18 18:24:31,427159,4,light_denim_blue,skinny_fit,184.0,31/32,0.93,0.06,0.01,0.0
4,427159006,trashed_skinny_jeans,men_jeans_ripped,39.99,2021-07-18 18:24:31,427159,5,dark_denim_blue,skinny_fit,184.0,31/32,0.93,0.06,0.01,0.0


# 3.1 Pratice time! (commands SQL)

In [18]:
query = """
    SELECT product_name, product_category, color_name, AVG(product_price) as "mean price"
    FROM showroom
    GROUP BY product_name, product_category, color_name
"""

# read db
df = pd.read_sql_query(query, conn)
df

,product_name,product_category,color_name,mean price
0,freefit®_slim_jeans,men_jeans_slim,black/no_fade_black,49.99
1,freefit®_slim_jeans,men_jeans_slim,dark_blue,49.99
2,freefit®_slim_jeans,men_jeans_slim,dark_denim_blue,49.99
3,freefit®_slim_jeans,men_jeans_slim,dark_gray,49.99
4,freefit®_slim_jeans,men_jeans_slim,gray,49.99
...,...,...,...,...
130,trashed_skinny_jeans,men_jeans_ripped,denim_blue/bleached,39.99
131,trashed_skinny_jeans,men_jeans_ripped,light_blue_washed_out,39.99
132,trashed_skinny_jeans,men_jeans_ripped,light_denim_blue,39.99
133,trashed_skinny_jeans,men_jeans_ripped,super_light_denim_blue,39.99


In [14]:
query = """
    SELECT product_category, AVG(product_price) as "mean price"
    FROM showroom
    GROUP BY product_category
    ORDER BY CAST ("mean price" as REAL) DESC
"""

# read db
df = pd.read_sql_query(query, conn)
df

,product_category,mean price
0,men_jeans_ripped,39.990000
1,men_jeans_slim,38.386226
2,men_jeans_regular,31.829080
3,men_jeans_relaxed,29.990000
4,men_jeans_skinny,28.155138
5,no_category,25.384737


In [17]:
query = """
    SELECT product_category, AVG(product_price) as "mean price", AVG(cotton), AVG(polyester), AVG(elastane), AVG(elasterell)
    FROM showroom
    GROUP BY product_category
    ORDER BY CAST ("mean price" as REAL) DESC
"""

# read db
df = pd.read_sql_query(query, conn)
df

,product_category,mean price,AVG(cotton),AVG(polyester),AVG(elastane),AVG(elasterell)
0,men_jeans_ripped,39.990000,0.955806,0.029032,0.015161,0.000000
1,men_jeans_slim,38.386226,0.950094,0.000000,0.019717,0.030189
2,men_jeans_regular,31.829080,0.975632,0.013793,0.010575,0.000000
3,men_jeans_relaxed,29.990000,1.000000,0.000000,0.000000,0.000000
4,men_jeans_skinny,28.155138,0.800275,0.182385,0.017339,0.000000
5,no_category,25.384737,0.982763,0.000000,0.012500,0.004737
